### Framework imports

In [ ]:
from noronha.tools.publish import Publisher
from noronha.tools.shortcuts import data_path, tmp_path

### Application imports

In [ ]:
import numpy as np
from sklearn import svm
from sklearn.externals import joblib
from sklearn.model_selection import StratifiedKFold, cross_val_score

### Training parameters

In [ ]:
gamma = 0.1
kernel = 'linear'
n_folds = 3

In [ ]:
params = {"gamma": gamma, "kernel": kernel, "n_folds": n_folds}

### Loading the dataset

In [ ]:
entries = np.genfromtxt(data_path('measures.csv'), delimiter=',')
classes = np.genfromtxt(data_path('species.csv'), dtype=str)

### Creating the estimator and folding strategy

In [ ]:
clf = svm.SVC(kernel=kernel, gamma=gamma)
fold = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=np.random.RandomState(19))

### Training

In [ ]:
score = cross_val_score(estimator=clf, X=entries, y=classes, cv=fold, n_jobs=-1)  # calculating metrics

metrics = dict(
    score_avg=score.mean(),
    score_var=np.sqrt(score.var())
)

clf.fit(entries, classes)  # definitive training

### Model publish

In [ ]:
joblib.dump(clf, tmp_path('clf.pkl'))

Publisher()(
    details=dict(
        params=params,
        metrics=metrics
    ),
    src_path=tmp_path()
)